In [8]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords


from sklearn.model_selection import train_test_split

In [9]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abhishekwaghchaure/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/abhishekwaghchaure/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
df = pd.read_csv('/Users/abhishekwaghchaure/Desktop/Datasets/email/emails.csv')
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [11]:
print(df.info())
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   file     517401 non-null  object
 1   message  517401 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB
None
(517401, 2)


In [12]:
df['message'][2]

"Message-ID: <24216240.1075855687451.JavaMail.evans@thyme>\nDate: Wed, 18 Oct 2000 03:00:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: leah.arsdall@enron.com\nSubject: Re: test\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: Leah Van Arsdall\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Dec2000\\Notes Folders\\'sent mail\nX-Origin: Allen-P\nX-FileName: pallen.nsf\n\ntest successful.  way to go!!!"

In [13]:
## Missing Values
df.isnull().sum()

file       0
message    0
dtype: int64

#### No missing Values Detected

In [14]:
## Extracting Email Headers and body
def split_email_content(email):
    headers, body = email.split('\n\n', 1) if '\n\n' in email else (email, "")
    return pd.Series([headers, body])

In [15]:
df[['headers', 'body']] =df['message'].apply(split_email_content)

In [16]:
df.head()

,file,message,headers,body
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast\n\n
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy,\n\n Can you send me a schedule of the s..."
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.


In [17]:
# print(df['message'][2])
print(df['body'][2])
print(df['headers'][2])

test successful.  way to go!!!
Message-ID: <24216240.1075855687451.JavaMail.evans@thyme>
Date: Wed, 18 Oct 2000 03:00:00 -0700 (PDT)
From: phillip.allen@enron.com
To: leah.arsdall@enron.com
Subject: Re: test
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Leah Van Arsdall
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf


In [18]:
df = df.drop(columns = ['message'])
print(f"Email split into headers and body")

Email split into headers and body


In [19]:
## Extract Usefull Headers
def extract_header_value(header_text, field):
    match = re.search(fr"{field}: (.+)", header_text, re.IGNORECASE)
    return match.group(1).strip() if match else None


In [20]:
df['subject'] = df['headers'].apply(lambda x: extract_header_value(x, 'Subject'))
df['from'] = df['headers'].apply(lambda x: extract_header_value(x, 'from'))
df['to'] = df['headers'].apply(lambda x: extract_header_value(x, 'to'))

In [21]:
df.iloc[1000]

file                              allen-p/all_documents/462.
headers    Message-ID: <1081797.1075855696183.JavaMail.ev...
body       Lucy,\n\nHere is the rentroll.\n\nMy only ques...
subject                                                 None
from                                 phillip.allen@enron.com
to                                stagecoachmama@hotmail.com
Name: 1000, dtype: object

In [22]:
## Cleaning Text
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

In [23]:
df['clean_body'] = df['body'].apply(clean_text)

In [24]:
df['clean_body'][3]

'Randy Can you send me a schedule of the salary and level of everyone in the scheduling group Plus your thoughts on any changes that need to be made Patti S for example Phillip'

In [25]:
df['body'][3]

'Randy,\n\n Can you send me a schedule of the salary and level of everyone in the \nscheduling group.  Plus your thoughts on any changes that need to be made.  \n(Patti S for example)\n\nPhillip'

In [26]:
def preprocess_text(text):
    corpus = []
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in set(stopwords.words('english'))]
    corpus.append(words)
    return corpus

In [27]:
df['processed_body']  = df['clean_body'].apply(preprocess_text)

In [28]:
df['processed_body']

0                                        [[Here, forecast]]
1         [[Traveling, business, meeting, takes, fun, tr...
2                             [[test, successful, way, go]]
3         [[Randy, Can, send, schedule, salary, level, e...
4                            [[Lets, shoot, Tuesday, 1145]]
                                ...                        
517396    [[This, trade, OILSPECHEDGENG, John, Lavoratos...
517397    [[Some, position, Alberta, Term, book, I, send...
517398    [[2, Original, Message, From, Doucet, Dawn, Se...
517399    [[Analyst, Rank, Stephane, Brodeur, 1, Chad, C...
517400    [[think, YMCA, class, people, recovering, hear...
Name: processed_body, Length: 517401, dtype: object

In [29]:
## Extracting Dates from Email Headers
def extract_date(headers):
    match = re.search(r"Date: (.+)", headers)
    if match:
        return match.group(1).strip()
    return None

In [30]:
df['date'] = df['headers'].apply(extract_date)
df['date'].head()

0    Mon, 14 May 2001 16:39:00 -0700 (PDT)
1     Fri, 4 May 2001 13:51:00 -0700 (PDT)
2    Wed, 18 Oct 2000 03:00:00 -0700 (PDT)
3    Mon, 23 Oct 2000 06:13:00 -0700 (PDT)
4    Thu, 31 Aug 2000 05:07:00 -0700 (PDT)
Name: date, dtype: object

In [31]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['date'].head()

/var/folders/vt/hgv58_ws0rjfsg_1d8fcxgww0000gn/T/ipykernel_1236/739734227.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')
/var/folders/vt/hgv58_ws0rjfsg_1d8fcxgww0000gn/T/ipykernel_1236/739734227.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


0    2001-05-14 16:39:00-07:00
1    2001-05-04 13:51:00-07:00
2    2000-10-18 03:00:00-07:00
3    2000-10-23 06:13:00-07:00
4    2000-08-31 05:07:00-07:00
Name: date, dtype: object

In [38]:
df = df.sort_values(by=['subject', 'date']).reset_index(drop=True)
df['date'].head()

0    2002-02-01 11:25:00-08:00
1    2002-02-01 11:28:51-08:00
2    2002-02-01 11:32:28-08:00
3    2001-12-04 11:49:46-08:00
4    2001-03-19 03:17:00-08:00
Name: date, dtype: object

In [33]:
# Create a "response" column using the next email in the same thread
df['response'] = df.groupby('subject')['processed_body'].shift(-1)

# Drop rows where no response exists (e.g., last email in the thread)
df = df.dropna(subset=['response'])

# Display sample rows to verify
df[['subject', 'processed_body', 'response']].head()

,subject,processed_body,response
0,!!! OATI Etag 1.7 Minimum Requirements !!!,"[[Time, running, short, Is, company, prepared,...","[[Time, running, short, Is, company, prepared,..."
1,!!! OATI Etag 1.7 Minimum Requirements !!!,"[[Time, running, short, Is, company, prepared,...","[[Time, running, short, Is, company, prepared,..."
2,!!! OATI Etag 1.7 Minimum Requirements !!!,"[[Time, running, short, Is, company, prepared,...","[[Time, running, short, Is, company, prepared,..."
4,!!!!!!!!!!!GONE.SCR VIRUS Warning!!!!!!!!!!!11,"[[Immediately, delete, DO, NOT, OPEN, email, F...","[[Immediately, delete, DO, NOT, OPEN, email, F..."
7,""" Henwood's Rationalizing Midwest Power Market...","[[HENWOOD, ANNOUNCES, A, MAJOR, NEW, RELEASE, ...","[[HENWOOD, ANNOUNCES, A, MAJOR, NEW, RELEASE, ..."


In [34]:
df.shape

(339356, 10)

In [35]:
df

,file,headers,body,subject,from,to,clean_body,processed_body,date,response
0,semperger-c/deleted_items/46.,Message-ID: <30978077.1075841544706.JavaMail.e...,Time is running very short. Is your company p...,!!! OATI Etag 1.7 Minimum Requirements !!!,frank.billington@oatiinc.com,cara.semperger@enron.com,Time is running very short Is your company pre...,"[[Time, running, short, Is, company, prepared,...",2002-02-01 11:25:00-08:00,"[[Time, running, short, Is, company, prepared,..."
1,king-j/deleted_items/19.,Message-ID: <15244269.1075840797931.JavaMail.e...,Time is running very short. Is your company p...,!!! OATI Etag 1.7 Minimum Requirements !!!,frank.billington@oatiinc.com,jeff.king@enron.com,Time is running very short Is your company pre...,"[[Time, running, short, Is, company, prepared,...",2002-02-01 11:28:51-08:00,"[[Time, running, short, Is, company, prepared,..."
2,platter-p/inbox/43.,Message-ID: <394365.1075841413683.JavaMail.eva...,Time is running very short. Is your company p...,!!! OATI Etag 1.7 Minimum Requirements !!!,frank.billington@oatiinc.com,phillip.platter@enron.com,Time is running very short Is your company pre...,"[[Time, running, short, Is, company, prepared,...",2002-02-01 11:32:28-08:00,"[[Time, running, short, Is, company, prepared,..."
4,salisbury-h/inbox/196.,Message-ID: <19201127.1075841505530.JavaMail.e...,Immediately delete and DO NOT OPEN email \n \n...,!!!!!!!!!!!GONE.SCR VIRUS Warning!!!!!!!!!!!11,david.steiner@enron.com,center.dl-portland@enron.com,Immediately delete and DO NOT OPEN email From ...,"[[Immediately, delete, DO, NOT, OPEN, email, F...",2001-12-04 11:49:46-08:00,"[[Immediately, delete, DO, NOT, OPEN, email, F..."
7,kaminski-v/all_documents/1055.,Message-ID: <8575423.1075856206811.JavaMail.ev...,HENWOOD ANNOUNCES A MAJOR NEW RELEASE AND FUNC...,""" Henwood's Rationalizing Midwest Power Market...",cfarrell@hesinet.com,vkamins@ect.enron.com,HENWOOD ANNOUNCES A MAJOR NEW RELEASE AND FUNC...,"[[HENWOOD, ANNOUNCES, A, MAJOR, NEW, RELEASE, ...",2001-03-19 03:17:00-08:00,"[[HENWOOD, ANNOUNCES, A, MAJOR, NEW, RELEASE, ..."
...,...,...,...,...,...,...,...,...,...,...
498195,bass-e/discussion_threads/1074.,Message-ID: <19275406.1075854652806.JavaMail.e...,\nhttp://www.kgw.com/kgwnews/nationworld_story...,yummy,brian.hoskins@enron.com,"milson.mundim@enron.com, eric.bass@enron.com, ...",httpwwwkgwcomkgwnewsnationworld_storyhtmlStory...,[[httpwwwkgwcomkgwnewsnationworld_storyhtmlSto...,2000-12-04 02:17:00-08:00,[[httpwwwkgwcomkgwnewsnationworld_storyhtmlSto...
498207,farmer-d/_sent_mail/224.,Message-ID: <16526018.1075854163321.JavaMail.e...,"LA, \n\nCan you look into the Transport piece?...","zone transport, spot purchases",daren.farmer@enron.com,lauri.allen@enron.com,LA Can you look into the Transport piece We ne...,"[[LA, Can, look, Transport, piece, We, need, s...",2000-11-06 03:08:00-08:00,"[[LA, Can, look, Transport, piece, We, need, s..."
498208,farmer-d/all_documents/308.,Message-ID: <17252677.1075853973394.JavaMail.e...,"LA, \n\nCan you look into the Transport piece?...","zone transport, spot purchases",daren.farmer@enron.com,lauri.allen@enron.com,LA Can you look into the Transport piece We ne...,"[[LA, Can, look, Transport, piece, We, need, s...",2000-11-06 03:08:00-08:00,"[[LA, Can, look, Transport, piece, We, need, s..."
498209,farmer-d/discussion_threads/2584.,Message-ID: <19949877.1075854105248.JavaMail.e...,"LA, \n\nCan you look into the Transport piece?...","zone transport, spot purchases",daren.farmer@enron.com,lauri.allen@enron.com,LA Can you look into the Transport piece We ne...,"[[LA, Can, look, Transport, piece, We, need, s...",2000-11-06 03:08:00-08:00,"[[LA, Can, look, Transport, piece, We, need, s..."


In [43]:
# Saving the preprocessed data
df.to_csv("/Users/abhishekwaghchaure/Desktop/Datasets/email/preprocessed_emails.csv", index=False)
print("\nPreprocessed dataset saved as 'preprocessed_emails.csv'.")


Preprocessed dataset saved as 'preprocessed_emails.csv'.
